# Homework 3 - Ludek Cizinsky, 377297

In this homework, we are going to work with the transformer. There are three parts of this homework.

- In the first part, we are going to implement **positional encoding** and **self-attention**  and test them on a simple text dataset which contains around 100 sentences. We will use a small transformer in this task.

- In the second part, we will detect promoters from the DNA sequences. The main difference compared to the previous task is to tokenize the DNA sequence. Thus, our task here is to build the **tokenizer** to tokenize the DNA sequence. For the model, we will continue using the small transformer.

- In the third part, we will use a **foundation model** DNABERT to perform promoter detection. In this part, you do not need to train the transformer. Instead, you need to find and load the correct pre-trained model and then use it to get the embedding of the DNA sequence. Then, you will build a simple classifier to perform promoter detection based on the DNA embedding.

## 0. Initialization

Import the packages you are going to use here.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import random
import numpy as np
from torchmetrics.classification import BinaryF1Score

import ipywidgets as widgets
from types import SimpleNamespace
from utils import data, evaluation, models, visualization, text_exercise

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from bhtsne import tsne
import matplotlib.pyplot as plt

import math
import os

from tqdm import tqdm

from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

Set the hyperparameters.

In [ ]:
# Set seeds
seed = 128
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1. Positional Encoding and Self-Attention (7 pts)

### 1.1. Sinusoidal Positional Encoding (1 pt)

In this section, you are going to implement the sinusoidal positional encoding. The formula is as the following:

<div>
<img src="./imgs/positional embedding.png" width="400"/>
</div>

where $t$ is the desired position in the input and $\mathsf{\omega}_k$ follows:

<div>
<img src="./imgs/omega.png" width="200"/>
</div>

To see the details of sinusoidal positional encoding, you can check this [link](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/).

In [ ]:
class PositionalEmbedding(nn.Module):

    def __init__(self, max_position_embeddings, hidden_size, device):
        super().__init__()

        '''Construct the positional embedding.

        Args:
            max_position_embeddings (int): maximum length of the input - related to t in the previous formula
            hidden_size (int): encoding dimension - d in the previous formula
        '''

        # Account for odd hidden size dimension by assuming first we have even dimension
        # In the last step, we will just drop the last column
        is_odd = hidden_size % 2
        if is_odd:
            hidden_size += 1 

        # Compute weights for the positional embedding
        k = torch.arange(hidden_size//2, dtype=torch.float32, device=device)
        w = 1 / (10000 ** (2*k/hidden_size))

        # Compute the positional embedding
        t = torch.arange(max_position_embeddings, dtype=torch.float32, device=device)
        sin = torch.sin(torch.outer(t, w)) # n x d/2
        cos = torch.cos(torch.outer(t, w))  # n x d/2

        # Put the sin and cos together
        self.positional_embedding = torch.zeros((max_position_embeddings, hidden_size), device=device)
        self.positional_embedding[:, 0::2] = sin
        self.positional_embedding[:, 1::2] = cos

        # Drop the last column
        if is_odd:
            self.positional_embedding = self.positional_embedding[:, :-1]
        
    def forward(self, x):
        return self.positional_embedding
    
    def embedding(self):
        return self.positional_embedding

Here, you can visualize your positional encoding. If you implement everything correctly, you can get a figure that is similar to Figure 2 in this [link](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/).

In [ ]:
visualize_embedding, dimension_selector, max_len_selector = visualization.display_positional_encoding(PositionalEmbedding)
ui = widgets.HBox([max_len_selector, dimension_selector])  
out = widgets.interactive_output( visualize_embedding, {'max_len': max_len_selector, 'dimension': dimension_selector})
display(ui, out)

### 1.2. Self-Attention Mechanism (5 pts)

In this section, you are going to implement the self-attention mechanism. Please check the section 'Self-Attention in Detail' in this [link](https://jalammar.github.io/illustrated-transformer/) for the details of self-attention mechanism. (We encourage you to carefully go through the link since it is a very good tutorial for transformer.)

The specific steps will be provided in the comments of the following code. (The steps are only for reference. You do need to follow the steps if you have a better way to implement it.)

In [ ]:
class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads)
            )
        self.output_attentions = config.output_attentions
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
    ):

        # Resulting shape: (batch_size, sequence_length, all_head_size)
        mixed_key_layer = self.key(hidden_states)
        mixed_query_layer = self.query(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        # Resulting shape: (batch_size, num_attention_heads, sequence_length, attention_head_size)
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Resulting shape: (batch_size, num_attention_heads, sequence_length, sequence_length)
        # For each input token, we have unnormalized scores for all other tokens in the input sequence
        attention_scores = query_layer @ key_layer.transpose(-1, -2) / math.sqrt(self.attention_head_size)
        
        # Explanation of attention_mask: https://lukesalamone.github.io/posts/what-are-attention-masks/
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores
        attention_probs = F.softmax(attention_scores, dim=-1)

        # Apply dropout
        attention_probs = self.dropout(attention_probs)

        # (optional) Apply head mask if provided
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        # Reweight each token embeddings by the attention scores
        # Shape: (batch_size, num_attention_heads, sequence_length, attention_head_size)
        context_layer = attention_probs @ value_layer

        # Concatenate all the attention heads together
        head_outputs = torch.split(context_layer, 1, dim=1)
        context_layer = torch.cat(head_outputs, dim=-1).squeeze(1)

        # Get the output
        outputs = (context_layer, attention_probs) if self.output_attentions else (context_layer,)
        return outputs

Let's test your implementation using simple text data! First, let's load the data.

We use a small dataset in this homework for a shorter training time.

In [ ]:
# ChatGPT generated text data about BERT
text = text_exercise.get()
sentences_df, vocab = data.to_sentence_df(text)

After loading the data, you can train your model. Here we train our model using masked token prediction.

Hint: The final model accuracy should be higher than 0.9.

In [ ]:
text_max_len = 11

text_config = SimpleNamespace(
        vocab_size=len(vocab),
        hidden_size=60,
        max_position_embeddings=text_max_len,
        type_vocab_size=1,
        layer_norm_eps=1e-12,
        hidden_dropout_prob=0.0,
        attention_probs_dropout_prob=0.0,
        num_attention_heads=2,
        hidden_act="gelu",
        intermediate_size=160,
        num_hidden_layers=1,
        is_decoder=False,
        output_attentions=True,
        output_hidden_states=False,
        pruned_heads = {},
        initializer_range=0.02,
        device="cpu"
    )

tokenizer = data.TextTokenizer(vocab)
input_ids, segment_ids, masked_lm_labels, labels_idx, labels, attention_masks = data.generate_masked_data(sentences_df, tokenizer, k=1, max_len=text_max_len, noise_rate=0.4)

model = models.BertForMaskedLM(config=text_config, positional_embedding=PositionalEmbedding, attention=BertSelfAttention)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
print(f"Number of trainable model parameters: {models.number_of_model_parameters(model)}")

for epoch in range(200):
    optimizer.zero_grad()
    loss, outputs, attentions = model(
        input_ids=input_ids,
        token_type_ids=segment_ids,
        masked_lm_labels=masked_lm_labels,
        attention_mask=attention_masks
    )
    if (epoch + 1) % 20 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

print(f"Final model accuracy: {evaluation.masked_label_accuracy(labels, labels_idx, outputs.data)}")

### 1.3. Visualize Attention (1 pt)

Here, you can visualize the self-attention. 

Question: Can you interpret the visualization of the self-attention?

**Answer:** In each row, we have weights that denote the importance of each token in the input sequence for the current token (ith token). The darker the color, the more important the token is. Generally, we would expect the model to pay attention to tokens close to the current token, making the diagonal of the matrix darker. However, we observe that our model focuses attention on a few specific words rather arbitrarily (the column(s) appear dark). The most likely explanation for such behavior is the small size of the dataset. The model has likely memorized the training data and does not generalize well.

In [ ]:
visualize_attention, sample_id_selector = visualization.display_attantion(attentions=attentions, input_ids=input_ids, tokenizer=tokenizer)
widgets.interactive(visualize_attention, sample_id=sample_id_selector)

### 1.4. Train on small Wikitext Dataset

Here, you can **optionally** test your model on the smallest wikitext dataset. You should get an test accuracy around 0.4 after training 50 epochs.

This part is only for you to test your code. You can choose to run it or not. It takes around 1 hour to train the model for 50 epochs on the smallest wikitext dataset with Google Colab.

In [ ]:
# text_exercise.train_wikitext(device, positional_embedding=PositionalEmbedding, attention=BertSelfAttention)

## 2. Promoter detection (7 pts)

In this section, we detect promoter in DNA sequence.

A promoter is a region of DNA upstream of a gene where relevant proteins (such as RNA polymerase and transcription factors) bind to initiate transcription of that gene. Promoter detection is to identify if there are promoter regions in the given DNA sequence. We have covered this in the lecture. (If you are interested in the promoter, you can check this [link](https://www.genome.gov/genetics-glossary/Promoter) for more details.)

Here, we use a transformer and a classifier. The transformer first embeds the DNA sequences into features, and then the classifier detects the promoter based on the features.

The main difference between text and DNA sequence is how to tokenize the sequence. Thus, you need to implement a tokenizer for the DNA sequence.

### 2.1. DNA Tokenizer (1 pts)

Here, you will implement the DNA tokenizer the same as in DNABERT. Please check this [paper](https://academic.oup.com/bioinformatics/article/37/15/2112/6128680) for implementation details. Also, you need to check the data type and shape for both input and output.

In [ ]:
class DNATokenizer(data.Tokenizer):
    def __init__(self, k, vocab, unknown="[UNK]"):
        super().__init__(vocab, unknown)

        # self.k is the k of k-mers
        self.k = k

    def _parse_text(self, text):
        # Split the text into k-mers and add special tokens to the start and end of the sequence
        # [CLS] is denoting a special token that can be used later for classification, the idea behind
        # is that it should represent the whole sequence
        # [SEP] is denoting the end of the sequence
        n = len(text)
        return ['[CLS]'] + [text[i:i + self.k] for i in range(n - self.k + 1)] + ['[SEP]']

### 2.2. Test BERT on DNA Sequence

In this section, you will train BERT on DNA sequence to learn the embedding of DNA sequence. The code is provided below and you do not need to write anything.

Hint: the final evaluation accuracy should be higher than 0.2.

In [ ]:
kmer = 3
mask_length = kmer
VOCAB_3MER = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "AAA", "AAT", "AAC", "AAG", "ATA", "ATT", "ATC", "ATG", "ACA", "ACT", "ACC", "ACG", "AGA", "AGT", "AGC", "AGG", "TAA", "TAT", "TAC", "TAG", "TTA", "TTT", "TTC", "TTG", "TCA", "TCT", "TCC", "TCG", "TGA", "TGT", "TGC", "TGG", "CAA", "CAT", "CAC", "CAG", "CTA", "CTT", "CTC", "CTG", "CCA", "CCT", "CCC", "CCG", "CGA", "CGT", "CGC", "CGG", "GAA", "GAT", "GAC", "GAG", "GTA", "GTT", "GTC", "GTG", "GCA", "GCT", "GCC", "GCG", "GGA", "GGT", "GGC", "GGG" ]

raw_training_data = data.load_csv("./data/train.csv")
raw_test_data = data.load_csv("./data/test.csv")

dna_max_len = 300 # NB: change to account for the two special tokens
batch_size = 128
max_dna_mask = 100
dataset_size = 1000
num_layers = 3
num_heads = 6
dna_config = SimpleNamespace(
        vocab_size=len(VOCAB_3MER),
        hidden_size=60,
        max_position_embeddings=dna_max_len,
        type_vocab_size=1,
        layer_norm_eps=1e-12,
        hidden_dropout_prob=0.0,
        attention_probs_dropout_prob=0.0,
        num_attention_heads=num_heads,
        hidden_act="gelu",
        intermediate_size=160,
        num_hidden_layers=num_layers,
        is_decoder=False,
        output_attentions=True,
        output_hidden_states=True,
        pruned_heads = {},
        initializer_range=0.02,
        device=device
    )

tokenizer = DNATokenizer(k=kmer, vocab=VOCAB_3MER)
input_ids, segment_ids, masked_lm_labels, labels_idx, labels, attention_masks = data.generate_masked_data(raw_training_data, tokenizer, max_len=dna_max_len, max_mask=max_dna_mask, k=mask_length, mask_rate=0.05, max_size=dataset_size)
test_input_ids, test_segment_ids, test_masked_lm_labels, test_labels_idx, test_labels, test_attention_masks = data.generate_masked_data(raw_test_data, tokenizer, max_len=dna_max_len, max_mask=max_dna_mask, k=mask_length, mask_rate=0.05, max_size=dataset_size)

train_dataset = TensorDataset(input_ids, segment_ids, masked_lm_labels, labels_idx, labels, attention_masks)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_segment_ids, test_masked_lm_labels, test_labels_idx, test_labels, test_attention_masks)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train the model
if not os.path.exists('models/bert_longmask.pth'):
  print("Training the model...")
  model = models.BertForMaskedLM(config=dna_config, positional_embedding=PositionalEmbedding, attention=BertSelfAttention).to(device)
  optimizer = optim.AdamW(model.parameters(), lr=0.002)
  print(f"Number of trainable model parameters: {models.number_of_model_parameters(model)}")

  for epoch in range(50):
    total_train_loss = 0
    model.train()
    for batch_input_ids, batch_segment_ids, batch_masked_lm_labels, _, _, batch_attention_mask in train_loader:
      optimizer.zero_grad()
      loss, outputs, hidden_states, _ = model(
          input_ids=batch_input_ids.to(device),
          token_type_ids=batch_segment_ids.to(device),
          masked_lm_labels=batch_masked_lm_labels.to(device),
          attention_mask=batch_attention_mask.to(device)
      )
      loss.backward()
      optimizer.step()
      total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    if (epoch + 1) % 10 == 0:
      model.eval()
      total_eval_loss = 0
      for batch_input_ids, batch_segment_ids, batch_masked_lm_labels, _, _, batch_attention_mask in test_loader:
        with torch.no_grad():
          loss, outputs, hidden_states, _ = model(
            input_ids=batch_input_ids.to(device),
            token_type_ids=batch_segment_ids.to(device),
            masked_lm_labels=batch_masked_lm_labels.to(device),
            attention_mask=batch_attention_mask.to(device)
          )
          if batch_attention_mask.sum() - torch.numel(batch_attention_mask) > 0 :
            print("found patting", batch_attention_mask.sum())
          total_eval_loss += loss.item()
      avg_eval_loss = total_eval_loss / len(test_loader)
      print('Epoch:', '%04d' % (epoch + 1), 'train cost =', '{:.6f}'.format(avg_train_loss), 'eval cost =', '{:.6f}'.format(avg_eval_loss))

  # Save the trained model
  os.makedirs('models', exist_ok=True)
  torch.save(model, 'models/bert_longmask.pth')

# Load the trained model
else:
  print("Loading the trained model...")
  model = torch.load('models/bert_longmask.pth')
  print(f"Number of trainable model parameters: {models.number_of_model_parameters(model)}")

average_train_acc, _ = evaluation.model_masked_label_accuracy(model, train_loader, device)
average_test_acc, last_test_attention = evaluation.model_masked_label_accuracy(model, test_loader, device)
print('Train Acc =', '{:.6f}'.format(average_train_acc), 'Eval Acc =', '{:.6f}'.format(average_test_acc))

### 2.3. Visualize the Attentions (1 pt)

Here, you can visualize the self-attention. 

Question: compare the visualization to Section 1.3, what can you find here? How do you explain it?

**Answer:** The main difference now is that we can observe a clear diagonal pattern for the majority of heads in each layer, indicating that the model is paying attention to tokens close to the current token. This change can likely be attributed to the training dataset size, which is 10 times larger in the latter case, making overfitting more challenging. However, for certain layers and heads, we still notice that the model is for instance focusing on tokens that are far away from the current token. This behavior aligns with our expectations, as we intend for each head to learn different features and, consequently, focus on different parts of the input sequence.

In [ ]:
visualize_attention, sample_id_selector, layer_selector, head_selector = visualization.display_multi_attantion(attentions=last_test_attention, tokenizer=tokenizer, input_ids=input_ids,  layers=range(1, num_layers+1),  heads=range(1, num_heads+1))
ui = widgets.HBox([sample_id_selector, layer_selector, head_selector])  
out = widgets.interactive_output(visualize_attention, {'sample_id': sample_id_selector, 'layer': layer_selector, 'head': head_selector})
display(ui, out)

### 2.4. Use your pretrained model for promoter detection (5 pts)

You already have the embeddings for the DNA sequence. Now, you are going to build a classifier based on the DNA embeddings. The classifier is to perform promoter detection. Specifically, the DNA sequence will be classified into *'contains promoter'* or *'does not contain promoter'*.

Hint: 
- We now want to annotate data (get the label for each sample), not predict masked data anymore!
- You can reuse some parts of the code in the previous sections, e.g. dataloader and training pipeline in Section 2.2.
- If you implement the previous section correctly (the Eval Acc > 0.2 in Section 2.2), you already have an pre-trained object named 'model' of class models.BertForMaskedLM. You can directly use it.
- The evaluation accuracy of this task should be around 0.6.

In [ ]:
# Define a binary classifier
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedder):
        super().__init__()
        self.embedder = embedder
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):

      # Get the output embeddings of BertForMaskedLM
      hidden_states = self.embedder(x)[1]

      # Get the output embeddings
      outputs = hidden_states[-1]

      # Use the [CLS] token embedding
      x = outputs[:, 0, :]

      # Run the classifier
      x = self.linear1(x)
      x = self.activation(x)
      x = self.linear2(x)

      return x.squeeze(1)

In [ ]:
# Load the data (without masks indeed)
input_ids, labels = data.generate_labeled_data(raw_training_data, tokenizer, max_len=dna_max_len, max_size=dataset_size)
test_input_ids, test_labels = data.generate_labeled_data(raw_test_data, tokenizer, max_len=dna_max_len, max_size=dataset_size)

# Convert labels to float
labels = labels.float()
test_labels = test_labels.float()

# Define data loaders
train_dataset = TensorDataset(input_ids, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Train the model first
criterion = nn.BCEWithLogitsLoss()
if not os.path.exists('models/promoter_clf.pth'):

  print("Training the model...")
  # Freeze the weights of the embedder model
  for param in model.parameters():
      param.requires_grad = False

  # Define the classifier
  clf = BinaryClassifier(input_size=60, hidden_size=30, output_size=1, embedder=model).to(device)
  clf_optimizer = optim.AdamW(clf.parameters(), lr=1e-2)

  # Train the classifier
  clf.train()
  for epoch in range(50):

    # Training
    total_train_loss = 0
    preds = []
    labs = []
    for batch_input_ids, batch_labels in train_loader:

      # Reset the gradients
      clf_optimizer.zero_grad()

      # Get the output of the classifier
      batch_output = clf(batch_input_ids)

      # Compute the loss
      loss = criterion(batch_output.to(device), batch_labels.to(device))

      # Add prediction and labels
      batch_output = F.sigmoid(batch_output)
      bpred = (batch_output > 0.5).float().tolist()
      blabs = batch_labels.tolist()
      preds.extend(bpred)
      labs.extend(blabs)

      # Backpropagation
      loss.backward()
      clf_optimizer.step()
      total_train_loss += loss.item()

    # Evaluate the epoch
    preds, labs = np.array(preds), np.array(labs)
    avg_train_loss = total_train_loss / len(train_loader)
    train_acc = np.mean(preds == labs)
    if (epoch + 1) % 10 == 0:
      print('Epoch:', '%04d' % (epoch + 1), 'train cost =', '{:.6f}'.format(avg_train_loss), 'train acc =', '{:.6f}'.format(train_acc))

  # Save the trained model
  os.makedirs('models', exist_ok=True)
  torch.save(clf, 'models/promoter_clf.pth')

# Load the trained model
else:
  print("Loading the trained model...")
  clf = torch.load('models/promoter_clf.pth')

# Evaluation
total_test_loss = 0
preds = []
labs = []
clf.eval()
for batch_input_ids, batch_labels in test_loader:

  # Get the output of the classifier
  batch_output = clf(batch_input_ids)

  # Compute the loss
  loss = criterion(batch_output.to(device), batch_labels.to(device))

  # Add prediction and labels
  batch_output = F.sigmoid(batch_output)
  bpred = (batch_output > 0.5).float().tolist()
  blabs = batch_labels.tolist()
  preds.extend(bpred)
  labs.extend(blabs)

  total_test_loss += loss.item()

preds, labs = np.array(preds), np.array(labs)
avg_test_loss = total_test_loss / len(test_loader)
test_acc = np.mean(preds == labs)
print('Test cost =', '{:.6f}'.format(avg_test_loss), 'test acc =', '{:.6f}'.format(test_acc))


### 2.5. Additional question (1 pt)

Now we change mask_length = 1 (already changed, you do not need to implement anything).
Let's run the code below and check the accuracy.

Question: What is the final masked token prediction accuracy? How do you explain this?

**Answer**: The final prediction accuracy is high, reaching 92%. This robust performance is attributed to the simplicity of the prediction task when masking only one k-mer. To illustrate, consider the sequence ABCDEF, tokenized into 3-mers as [ABC, BCD, CDE, DEF]. If we mask a single k-mer, let's say BCD, we retain access to the surrounding k-mers, simplifying the prediction task. On the other hand, masking multiple k-mers, such as BCD and CDE, introduces additional complexity. In this scenario, each masked k-mer has access to only one direct neighbor, making the prediction task inherently more challenging.

In [ ]:
kmer = 3
mask_length = 1

dna_max_len = 300
batch_size = 128
max_dna_mask = 100
dataset_size = 1000
num_layers = 3
num_heads = 6
dna_config = SimpleNamespace(
        vocab_size=len(VOCAB_3MER),
        hidden_size=60,
        max_position_embeddings=dna_max_len,
        type_vocab_size=1,
        layer_norm_eps=1e-12,
        hidden_dropout_prob=0.0,
        attention_probs_dropout_prob=0.0,
        num_attention_heads=num_heads,
        hidden_act="gelu",
        intermediate_size=160,
        num_hidden_layers=num_layers,
        is_decoder=False,
        output_attentions=True,
        output_hidden_states=True,
        pruned_heads = {},
        initializer_range=0.02,
        device=device
    )

tokenizer = DNATokenizer(k=3, vocab=VOCAB_3MER)
input_ids, segment_ids, masked_lm_labels, labels_idx, labels, attention_masks = data.generate_masked_data(raw_training_data, tokenizer, max_len=dna_max_len, max_mask=max_dna_mask, k=mask_length, mask_rate=0.05, max_size=dataset_size)
test_input_ids, test_segment_ids, test_masked_lm_labels, test_labels_idx, test_labels, test_attention_masks = data.generate_masked_data(raw_test_data, tokenizer, max_len=dna_max_len, max_mask=max_dna_mask, k=mask_length, mask_rate=0.05, max_size=dataset_size)

train_dataset = TensorDataset(input_ids, segment_ids, masked_lm_labels, labels_idx, labels, attention_masks)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_segment_ids, test_masked_lm_labels, test_labels_idx, test_labels, test_attention_masks)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train the model
if not os.path.exists('models/bert_shortmask.pth'):
  print("Training the model...")
  model = models.BertForMaskedLM(config=dna_config, positional_embedding=PositionalEmbedding, attention=BertSelfAttention).to(device)
  optimizer = optim.AdamW(model.parameters(), lr=0.002)
  print(f"Number of trainable model parameters: {models.number_of_model_parameters(model)}")

  for epoch in range(50):
    total_train_loss = 0
    model.train()
    for batch_input_ids, batch_segment_ids, batch_masked_lm_labels, _, _, batch_attention_mask in train_loader:
      optimizer.zero_grad()
      loss, outputs, hidden_states, _ = model(
          input_ids=batch_input_ids.to(device),
          token_type_ids=batch_segment_ids.to(device),
          masked_lm_labels=batch_masked_lm_labels.to(device),
          attention_mask=batch_attention_mask.to(device)
      )
      loss.backward()
      optimizer.step()
      total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    if (epoch + 1) % 10 == 0:
      model.eval()
      total_eval_loss = 0
      for batch_input_ids, batch_segment_ids, batch_masked_lm_labels, _, _, batch_attention_mask in test_loader:
        with torch.no_grad():
          loss, outputs, hidden_states, _ = model(
            input_ids=batch_input_ids.to(device),
            token_type_ids=batch_segment_ids.to(device),
            masked_lm_labels=batch_masked_lm_labels.to(device),
            attention_mask=batch_attention_mask.to(device)
          )
          if batch_attention_mask.sum() - torch.numel(batch_attention_mask) > 0 :
            print("found patting", batch_attention_mask.sum())
          total_eval_loss += loss.item()
      avg_eval_loss = total_eval_loss / len(test_loader)
      print('Epoch:', '%04d' % (epoch + 1), 'train cost =', '{:.6f}'.format(avg_train_loss), 'eval cost =', '{:.6f}'.format(avg_eval_loss))

  # Save the trained model
  os.makedirs('models', exist_ok=True)
  torch.save(model, 'models/bert_shortmask.pth')

else:
  print("Loading the trained model...")
  model = torch.load('models/bert_shortmask.pth')
  print(f"Number of trainable model parameters: {models.number_of_model_parameters(model)}")

average_train_acc, _ = evaluation.model_masked_label_accuracy(model, train_loader, device)
average_test_acc, last_test_attention = evaluation.model_masked_label_accuracy(model, test_loader, device)
print('Train Acc =', '{:.6f}'.format(average_train_acc), 'Eval Acc =', '{:.6f}'.format(average_test_acc))

## 3. Using foundation model (5 pts)

### 3.1. Introduction

In this section, we aim to use a foundation model, DNABERT, to perform promoter detection.
A foundation model is a model pretrained on large datasets. Foundation models serve as the foundational building blocks upon which various applications can be constructed.

Here, we use DNABERT as the foundation model. We first apply it on DNA sequence to get the embedding. Then, we train a classifier on the embedding as in Section 2. Please follow this [link](https://github.com/Zhihan1996/DNABERT_2) to load the foundation model.

### 3.2. Implementation

**Consider this situation:** You get a dataset about promoter detection, and you build your model to perform the task as in Section 2. However, the performance is not good since the model is not strong enough. Suddenly, you think we can use a large pre-trained model to embed DNA sequences. Then, you search online and find the pre-trained model [DNABERT](https://github.com/Zhihan1996/DNABERT_2). Now, you want to perform promoter detection using the pre-trained DNABERT.

There is no coding framework in this section. Just make things work (get good test accuracy) using the pre-trained model!

Hint: 
- We encourage you to create a **new environment** following the instructions of Section 3 in this [link](https://github.com/Zhihan1996/DNABERT_2). (When you face the error "The model class you are passing has a config_class attribute that is not consistent with the config class you passed ...", creating a new environment can save you.)
- Section 4 in this [link](https://github.com/Zhihan1996/DNABERT_2) shows you how to load and use the pre-trained foundation model.

a. Load the dataset.

In [ ]:
# Load raw data
raw_training_data = data.load_csv("./data/train.csv")
raw_test_data = data.load_csv("./data/test.csv")

# Define the tokenizer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True) 

# Define training hyperparameters
dna_max_len = 298
dataset_size = 200

# Load the data
input_ids, labels = data.generate_labeled_data(raw_training_data, tokenizer, max_len=dna_max_len, max_size=dataset_size)
test_input_ids, test_labels = data.generate_labeled_data(raw_test_data, tokenizer, max_len=dna_max_len, max_size=dataset_size)

# Define data loaders
train_dataset = TensorDataset(input_ids, labels)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

test_dataset = TensorDataset(test_input_ids, test_labels)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

b. Get the embeddings of the DNA sequences using pretrained model.

Hint: 
- This step can take some time. Thus, you can start with a small sample size, and then increase it when you have made sure that everything works correctly.
- After getting the embeddings, you can save them so that you can directly load them next time without running the foundation model.

In [ ]:
# Define helper functions
def save_embeddings(loader, model, path):
    # Save the embeddings and labels as pytorch tensors
    labels, embeddings = [], []
    for data in tqdm(loader):
        # Get the input 
        input_ids, input_labels = data

        # Get the output
        hidden_states = model(input_ids)[0]
        
        # Aggregate the embedding usig mean pooling
        embedding_mean = torch.mean(hidden_states[0], dim=0)

        # Add the embedding and label
        embeddings.append(embedding_mean)
        labels.append(input_labels)

    # Convert to tensors
    embeddings = torch.stack(embeddings)
    labels = torch.stack(labels)

    # Save
    torch.save(embeddings, os.path.join(path, f"x.pth"))
    torch.save(labels, os.path.join(path, f"y.pth"))

def load_embeddings(path):
    # Load the embeddings and labels
    embeddings = torch.load(os.path.join(path, f"x.pth"))
    labels = torch.load(os.path.join(path, f"y.pth"))
    return embeddings, labels

In [ ]:
# Load the pretrained model
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to(device)

# Turn off the gradients
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Save the training embeddings
train_path = os.path.join("embeddings", "dnabert", "train")
if not os.path.exists(train_path):
    # First create the path
    os.makedirs(train_path, exist_ok=True)

    # Save the embeddings 
    print("Saving dnabert train embeddings")
    save_embeddings(train_loader, model, train_path)

# Save the test embeddings
test_path = os.path.join("embeddings", "dnabert", "test")
if not os.path.exists(test_path):
    # First create the path
    os.makedirs(test_path, exist_ok=True)

    # Save the embeddings
    print("Saving dnabert test embeddings")
    save_embeddings(test_loader, model, test_path)

In [ ]:
# Define custom torch dataset
class EmbeddingDataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __getitem__(self, index):
        return self.embeddings[index], self.labels[index]

    def __len__(self):
        return len(self.embeddings)

# Load the embeddings with labels
train_embeddings, train_labels = load_embeddings(train_path)
test_embeddings, test_labels = load_embeddings(test_path)

# Split the train into train and validation
train_embeddings, val_embeddings, train_labels, val_labels = train_test_split(train_embeddings, train_labels, test_size=0.5, random_state=seed)

# Define the datasets
train_dataset = EmbeddingDataset(train_embeddings, train_labels)
val_dataset = EmbeddingDataset(val_embeddings, val_labels)
test_dataset = EmbeddingDataset(test_embeddings, test_labels)

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
# Choose only a subset
n = min(len(train_embeddings), 1000)
emb, lab = train_embeddings[:n], train_labels[:n]

# Apply Standard Scaling
emb = (emb - emb.mean(dim=0)) / emb.std(dim=0)

# Reduce the dimensionality of the embeddings
pca = PCA(n_components=100)
reduced_data = pca.fit_transform(emb)

# Apply t-SNE on the reduced data
embedded_data = tsne(reduced_data, rand_seed=seed)

# Visualize the result
plt.scatter(embedded_data[:, 0], embedded_data[:, 1], c=lab, cmap=plt.cm.get_cmap("jet", 2), alpha=0.5)
plt.title(f't-SNE Visualization n={n}')
plt.show()

c. Train a classifier.

Hint: It is easy to overfit on the training set. Try to avoid overfitting.

In [ ]:
class DnaBertHead(nn.Module):
    def __init__(self, input_size, output_size, hidden_dimensions, dropout_rate=0.05, activation=nn.ReLU()):
        super(DnaBertHead, self).__init__()

        layers = []
        in_features = input_size

        for out_features in hidden_dimensions:
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.Dropout(p=dropout_rate))
            layers.append(activation)
            in_features = out_features

        layers.append(nn.Linear(hidden_dimensions[-1], output_size))
        
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze(1)

In [ ]:
def eval_loop(clf, test_loader, criterion, device, verbose=True):

    # Validation
    total_test_loss = 0
    preds = []
    labs = []
    clf.eval()
    with torch.no_grad():
        for batch_embeddings, batch_labels in test_loader:

            # Preprocess the labels
            batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.squeeze(1).float().to(device)

            # Get the output of the classifier
            batch_output = clf(batch_embeddings)

            # Compute the loss
            loss = criterion(batch_output, batch_labels)

            # Add prediction and labels
            batch_output = torch.sigmoid(batch_output)
            bpred = (batch_output > 0.5).float().tolist()
            blabs = batch_labels.tolist()
            preds.extend(bpred)
            labs.extend(blabs)

            total_test_loss += loss.item()

    # Evaluate the epoch
    preds, labs = np.array(preds), np.array(labs)
    avg_test_loss = total_test_loss / len(test_loader)
    test_acc = np.mean(preds == labs)

    # Print the epoch results
    if verbose:
        print('Test cost =', '{:.6f}'.format(avg_test_loss), 'test acc =', '{:.6f}'.format(test_acc))

    return avg_test_loss, test_acc

In [ ]:
def train_loop(clf, train_loader, val_loader, criterion, optimizer, device, epochs=100, patience=5, verbose=True):

    # Initialize the early stopping
    best_val_loss = np.inf
    best_val_acc = 0
    best_epoch = 0
    epochs_without_improvement = 0

    # Train the classifier
    clf.train()
    for epoch in range(epochs):
            
            # Training
            total_train_loss = 0
            preds = []
            labs = []
            for batch_embeddings, batch_labels in train_loader:
    
                # Preprocess the labels
                batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.squeeze(1).float().to(device)
    
                # Reset the gradients
                optimizer.zero_grad()
    
                # Get the output of the classifier
                batch_output = clf(batch_embeddings)
    
                # Compute the loss
                loss = criterion(batch_output, batch_labels)
    
                # Add prediction and labels
                batch_output = torch.sigmoid(batch_output)
                bpred = (batch_output > 0.5).float().tolist()
                blabs = batch_labels.tolist()
                preds.extend(bpred)
                labs.extend(blabs)
    
                # Backpropagation
                loss.backward()
                optimizer.step()
                total_train_loss += loss.item()
    
            # Evaluate the epoch
            preds, labs = np.array(preds), np.array(labs)
            avg_train_loss = total_train_loss / len(train_loader)
            train_acc = np.mean(preds == labs)
    
            # Validation
            avg_val_loss, val_acc = eval_loop(clf, val_loader, criterion, device, verbose=False)
    
            # Print the epoch results
            if verbose:
                print('Epoch:', '%04d' % (epoch + 1), 
                      'train cost =', '{:.6f}'.format(avg_train_loss), 'train acc =', '{:.6f}'.format(train_acc), 
                      'val cost =', '{:.6f}'.format(avg_val_loss), 'val acc =', '{:.6f}'.format(val_acc))
            
            # Check if the validation loss has improved
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_val_acc = val_acc
                best_epoch = epoch
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
            
            # Check if we should stop the training
            if epochs_without_improvement == patience:
                break

    # Print the best epoch results
    if verbose:
        print('Best epoch:', '%04d' % (best_epoch + 1), 'val cost =', '{:.6f}'.format(best_val_loss), 'val acc =', '{:.6f}'.format(best_val_acc))
    
    return best_val_loss, best_val_acc

In [ ]:
# Train the model first
criterion = nn.BCEWithLogitsLoss()
if True or not os.path.exists('models/promoter_clf2.pth'):

  print("Training the model...")

  # Define the classifier
  clf = DnaBertHead(input_size=768, output_size=1, hidden_dimensions=[30], dropout_rate=0.55, activation=nn.ReLU()).to(device)
  clf_optimizer = optim.AdamW(clf.parameters(), lr=1e-1, weight_decay=0.05)

  # Train the classifier
  best_val_loss, best_val_acc = train_loop(clf, train_loader, val_loader, criterion, clf_optimizer, device, epochs=100, patience=5)

  # Save the trained model
  os.makedirs("models", exist_ok=True)
  torch.save(clf, 'models/promoter_clf2.pth')

# Load the trained model
else:
  print("Loading the trained model...")
  clf = torch.load('models/promoter_clf2.pth')

# Evaluation
avg_test_loss, test_acc = eval_loop(clf, test_loader, criterion, device)

---